<a href="https://colab.research.google.com/github/TreeLiquid/Intro-to-ML/blob/main/Shigapov_4105_HW6_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
#Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import tensorflow as tf
import torch.nn as nn
import torch.optim as optim

In [97]:
from google.colab import drive
drive.mount('/content/drive')

#Obtain file data
filepath = '/content/drive/My Drive/Fall-2023/Intro-to-ML/Datasets/Housing.csv'
HD = pd.DataFrame(pd.read_csv(filepath))
HD.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [98]:
#Converting True/False to 1/0
vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']
varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

def binary_map(x):
  return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

HD[varlist] = HD[varlist].apply(binary_map)
HD.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1


In [99]:
#Splitting Dataset
X_train, X_test = train_test_split(HD, train_size= 0.8, test_size= 0.2, random_state= 0)

#Splitting Variables
y_train = X_train.pop("price")
y_test = X_test.pop("price")
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

X_train.head()

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
542,3620,2,1,1,1,0,0,0,0,0,0,-1
496,4000,2,1,1,1,0,0,0,0,0,0,-1
484,3040,2,1,1,0,0,0,0,0,0,0,-1
507,3600,2,1,1,1,0,0,0,0,0,0,-1
252,9860,3,1,1,1,0,0,0,0,0,0,0


In [100]:
#Scaling
scale = StandardScaler()

std_X_train = X_train
std_X_train[vars] = scale.fit_transform(std_X_train[vars])

std_X_test = X_test
std_X_test[vars] = scale.fit_transform(std_X_test[vars])

In [101]:
#Stacking
stacked_Train = np.c_[np.ones((len(y_train), 1)), std_X_train[vars]]
stacked_test = np.c_[np.ones((len(y_test), 1)), std_X_test[vars]]

In [102]:
#To Tensors
X_train_ten = torch.tensor(stacked_Train, dtype=torch.float)
X_test_ten = torch.tensor(stacked_test, dtype=torch.float)
y_train_ten = torch.tensor(y_train, dtype=torch.float)
y_test_ten = torch.tensor(y_test, dtype=torch.float)

In [103]:
#Model with One Hidden Layer
model1 = nn.Sequential(
    nn.Linear(13, 32),
    nn.Tanh(),
    nn.Linear(32, 1)
)

In [104]:
#Training Loop
def TL(epochs, optimizer, model, lossFN, X_train, X_test, y_train, y_test):
  for epoch in range(1, epochs + 1):
    pred_train = model(X_train)
    loss_train = lossFN(pred_train, y_train)

    pred_test = model(X_test)
    loss_test = lossFN(pred_test, y_test)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch == 0 or epoch % 500 == 0:
      print(f"Epoch: {epoch}, Training Loss: {loss_train.item()}, Validation Loss: {loss_test}")

In [105]:
#Training
optimizer = optim.SGD(model1.parameters(), lr= 0.01)
print("Training with a model with 1 hidden layer")

TL(
    epochs= 5000,
    optimizer= optimizer,
    model= model1,
    lossFN= nn.MSELoss(),
    X_train= X_train_ten,
    X_test= X_test_ten,
    y_train= y_train_ten,
    y_test= y_test_ten
)

Training with a model with 1 hidden layer
Epoch: 500, Training Loss: 3570451087360.0, Validation Loss: 2888026816512.0
Epoch: 1000, Training Loss: 3564346015744.0, Validation Loss: 2888976302080.0
Epoch: 1500, Training Loss: 3563251564544.0, Validation Loss: 2889369780224.0
Epoch: 2000, Training Loss: 3562933321728.0, Validation Loss: 2889534144512.0
Epoch: 2500, Training Loss: 3562755063808.0, Validation Loss: 2889605709824.0
Epoch: 3000, Training Loss: 3562619535360.0, Validation Loss: 2889641623552.0
Epoch: 3500, Training Loss: 3562507599872.0, Validation Loss: 2889662857216.0
Epoch: 4000, Training Loss: 3562413752320.0, Validation Loss: 2889677275136.0
Epoch: 4500, Training Loss: 3562334846976.0, Validation Loss: 2889688547328.0
Epoch: 5000, Training Loss: 3562269310976.0, Validation Loss: 2889698770944.0


In [106]:
#Model with 3 Hidden Layers
model3 = nn.Sequential(
    nn.Linear(13, 32),
    nn.Tanh(),
    nn.Linear(32, 64),
    nn.Tanh(),
    nn.Linear(64, 16),
    nn.Tanh(),
    nn.Linear(16, 1)
)

In [107]:
#Training
optimizer3 = optim.SGD(model3.parameters(), lr= 0.01)
print("Training with a model with 3 hidden layers")

TL(
    epochs= 5000,
    optimizer= optimizer3,
    model= model3,
    lossFN= nn.MSELoss(),
    X_train= X_train_ten,
    X_test= X_test_ten,
    y_train= y_train_ten,
    y_test= y_test_ten
)

Training with a model with 3 hidden layers
Epoch: 500, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0
Epoch: 1000, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0
Epoch: 1500, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0
Epoch: 2000, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0
Epoch: 2500, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0
Epoch: 3000, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0
Epoch: 3500, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0
Epoch: 4000, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0
Epoch: 4500, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0
Epoch: 5000, Training Loss: 3644181970944.0, Validation Loss: 2884491542528.0


In [108]:
#Model Complexity
total_params = sum(p.numel() for p in model1.parameters())
print(f"Total parameters (Model w/ 1 Hidden Layer): {total_params}")

total_params = sum(p.numel() for p in model3.parameters())
print(f"Total parameters (Model w/ 3 Hidden Layers): {total_params}")

Total parameters (Model w/ 1 Hidden Layer): 481
Total parameters (Model w/ 3 Hidden Layers): 3617
